In [7]:
import cv2
import numpy as np
import os
import time
import csv
from datetime import datetime
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Function to preprocess the image
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply Canny Edge Detection with adjusted thresholds
    edges = cv2.Canny(blurred, 30, 100)  # Adjusted thresholds for edge detection
    
    return edges

# Function to check if a line is horizontal or vertical
def is_horizontal_or_vertical(x1, y1, x2, y2):
    angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    return abs(angle) < 10 or abs(angle) > 80  # Allow a small tolerance for horizontal and vertical

def extend_line(x1, y1, x2, y2, length=1000):
    # Compute the direction vector
    dx = x2 - x1
    dy = y2 - y1
    length_current = np.sqrt(dx**2 + dy**2)
    dx_normalized = dx / length_current
    dy_normalized = dy / length_current

    # Extend the line in both directions
    x1_new = int(x1 - length * dx_normalized)
    y1_new = int(y1 - length * dy_normalized)
    x2_new = int(x2 + length * dx_normalized)
    y2_new = int(y2 + length * dy_normalized)

    return (x1_new, y1_new, x2_new, y2_new)

# Function to detect lines and calculate angle
def detect_flow_angle(image_path, output_image_path):
    img = cv2.imread(image_path)
    
    # Check if the image is loaded successfully
    if img is None:
        print(f"Error: Unable to load image {image_path}")
        return None

    # Preprocess the image
    edges = preprocess_image(img)
    
    # Probabilistic Hough Line Transform to detect lines with adjusted parameters
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=50, maxLineGap=10)
    angles = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            
            # Check if the line is horizontal or vertical
            if not is_horizontal_or_vertical(x1, y1, x2, y2):
                # Draw each valid line on the original image
                x1_new, y1_new, x2_new, y2_new = extend_line(x1, y1, x2, y2)
                cv2.line(img, (x1_new, y1_new), (x2_new, y2_new), (0, 255, 0), 2)
                #cv2.line(img, (x1+100, y1+100), (x2, y2), (0, 255, 0), 2)
                
                # Calculate the angle of the line
                angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))*(-10)
                angles.append(angle)

    # If lines are detected, calculate the average angle
    if angles:
        avg_angle = np.mean(angles)
    else:
        avg_angle = None

    # Draw the average angle as text on the image
    if avg_angle is not None:
        cv2.putText(img, f"Angle: {avg_angle:.2f} degrees", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

    # Save the processed image
    cv2.imwrite(output_image_path, img)

    return avg_angle

# Function to write angle and timestamp to CSV
def write_to_csv(csv_file, angle):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, angle])

# Watchdog event handler class
class ImageHandler(FileSystemEventHandler):
    def __init__(self, output_folder, csv_file):
        self.output_folder = output_folder
        self.csv_file = csv_file

    def on_created(self, event):
        # Process only if the new file is an image
        if event.is_directory:
            return
        if event.src_path.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            time.sleep(1)  # Wait for the image file to be fully written to disk

            # Define the path for saving the processed image
            output_image_path = os.path.join(self.output_folder, os.path.basename(event.src_path))

            # Process the image and get the angle
            angle = detect_flow_angle(event.src_path, output_image_path)
            if angle is not None:
                write_to_csv(self.csv_file, angle)
                print(f"Processed {event.src_path}, Angle: {angle}, Saved: {output_image_path}")

# Function to monitor a folder for new images
def monitor_folder(folder_path, output_folder, csv_file):
    event_handler = ImageHandler(output_folder, csv_file)
    observer = Observer()
    observer.schedule(event_handler, folder_path, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)  # Keep the script running
    except KeyboardInterrupt:
        observer.stop()

    observer.join()

if __name__ == "__main__":
    folder_to_watch = "/Users/hanna m/machinelearning/deep_learning/cv/temp-eda/images"  # Replace with your image folder
    output_folder = "/Users/hanna m/machinelearning/deep_learning/cv/temp-eda/outputs"  # Replace with your output folder
    output_csv = "output_angles.csv"

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Add headers to the CSV file
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Angle (degrees)"])

    # Start monitoring the folder
    monitor_folder(folder_to_watch, output_folder, output_csv)


Processed /Users/hanna m/machinelearning/deep_learning/cv/temp-eda/images\10.jpg, Angle: 21.874891902701194, Saved: /Users/hanna m/machinelearning/deep_learning/cv/temp-eda/outputs\10.jpg
Processed /Users/hanna m/machinelearning/deep_learning/cv/temp-eda/images\6.jpg, Angle: 9.543475255231158, Saved: /Users/hanna m/machinelearning/deep_learning/cv/temp-eda/outputs\6.jpg
